# Day 3: データの結合やデータ変換のためのSQL操作
Day 2に引き続き，Day 3では関係データベース上に格納された購買データを用いた演習を行う．
Day 3では，
* 複数の表データの結合
* 関係データベース上での数値計算
* 分析のためのデータの変換

のためのSQL操作について体験する．

なお，Day 2からDay 5の演習で用いるデータセットは，[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)で提供されているデータセットをSQLite形式に変換したものである．また，「ノック」は[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)に収録されている演習課題を，本授業向けにアレンジしたものである．「課題」は完全に山本が作成したものである．

---

## SQL利用環境セットアップ
SQLiteはWindows/Mac/LinuxどのOSでもインストールして利用することができる．本演習は環境構築の手間をできるだけ減らすために，Google Colaboratory上でSQLiteを用いる．

Googleにログインした状態で[本演習サイト](https://data-analytics2022.hontolab.org)の各回の「ドリル（模範解答なし）」のリンクをクリックすると，あなた用のGoogle Colaboratoyがブラウザ上で立ち上がり，各回のドリル・課題を行うためのページが表示される．
ページが表示されたら，以下のコードが書かれた箇所（セル）を実行しよう．すると，Google Colaboratory上でSQLiteを実行するための環境が用意される．

なお，Google Colaboratory上でコードを実行するには，以下の2つの方法がある：
* 「再生ボタン」をクリックする
* コードが書かれたセルをクリックし，Shift + Enterキーを押す

In [1]:
# 現在のSQLiteのバージョンを確認
!sqlite3 --version

# SQLiteの最新バージョンをインストールするために、apt-getを使用
!apt-get update
!apt-get install sqlite3

# SQLiteの新しいバージョンをインストールした後、バージョンを再度確認
!sqlite3 --version

/bin/bash: line 1: sqlite3: command not found
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [825 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,037

上記コードを実行したら，アップグレード版SQLiteを使うために，Google Colaboratoryを再起動させよう．
再起動の方法は，画面上部の`Runtime`から`Restart Runtime`をクリックする．

再起動が完了したら，コードを実行しよう．以下のコードはGoogle Colab（Jupyter）上でSQLを対話的に利用するためのおまじないである．

In [2]:
# ipython-sqlがインストールされていない場合
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00


英文が数行表示された後，SQLiteがインストールされる．

続いて，演習で用いる購買データが収められたSQLiteデータベースをダウンロードしよう． 以下のコードを実行すると，Google Colaboratory上にデータベースがダウンロードされる．

In [3]:
# SQLiteデータベース（data-analytics-lecture.db）のファイルをダウンロード
import os
if not os.path.exists('data/data-analytics-lecture.db'):
    !wget -P data https://github.com/shoji360/data-analytics-2024/raw/main/data/data-analytics-lecture.db

--2024-05-02 03:49:42--  https://github.com/hontolab-courses/data-analytics-2022/raw/main/data/data-analytics-lecture.db
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hontolab-courses/data-analytics-2022/main/data/data-analytics-lecture.db [following]
--2024-05-02 03:49:42--  https://raw.githubusercontent.com/hontolab-courses/data-analytics-2022/main/data/data-analytics-lecture.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41496576 (40M) [application/octet-stream]
Saving to: ‘data/data-analytics-lecture.db’

data-analytics-lect 100%[===================>]  39.57M   182MB/s    in 0.2s    



データベースがダウンロードできたら，データをロードしておこう．
以下のコードを実行すると，Google Colaboratory上でダウンロードしたSQLiteデータベースを操作できるようになる．

In [4]:
%load_ext sql
%sql sqlite:///data/data-analytics-lecture.db

なお，Google Colaboratoryはセッションを閉じると，環境が初期化され，以前に自前でインストールしたライブラリが削除される．
Google Colaboratoryを使って演習に取り組む際には，上記コードを毎回忘れず実行しよう．

---

In [5]:
# ipython-sqlがインストールされていない場合
!pip install ipython-sql

In [6]:
%load_ext sql
%sql sqlite:///data/data-analytics-lecture.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


---

## Google Colaboratory上でのSQLの実行方法

Google Colaboratoryでは，以下のようにセルの冒頭に`%%sql`と書き，2行目以降にSQL文を書くとSQLを実行することができる．
試しに以下のSQLを実行してみよう．

In [7]:
%%sql

SELECT
    name AS table_name
FROM
    sqlite_master
WHERE
    type = 'table';

 * sqlite:///data/data-analytics-lecture.db
Done.


table_name
category
customer
geocode
product
receipt
store


上記SQLは，データベースに格納されたテーブルの一覧を表示するものである．
ダウンロードしたSQLiteデータベース，つまり今回扱う杏森堂のデータベースには，**6つ**のテーブルが格納されていることが分かる．なお，対象データベースの構造が記された実体関連図（ER図）は[コチラ](https://github.com/shoji360/data-analytics-2024/raw/main/doc/ER-diagram.pdf)から確認することできる．

課題とドリルに取り組む前に，あと1つ，別のSQLを実行してみよう．
以下のSQLを実行すると，`receipt`テーブルの最初の10件を表示することができる
（SQL文に`LIMIT`を付けないと，テーブル内の全データが表示されてしまうので注意）．

In [8]:
%%sql

SELECT * FROM receipt LIMIT 10;

 * sqlite:///data/data-analytics-lecture.db
Done.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20170101,1483228800,S12013,1112,1,ZZ000000000000,P090102011,1,180
20170101,1483228800,S12013,1142,2,ZZ000000000000,P070901024,1,198
20170101,1483228800,S12013,1172,2,ZZ000000000000,P070202007,1,138
20170101,1483228800,S12014,1152,1,CS014411000048,P080401010,1,110
20170101,1483228800,S12014,1152,2,CS014411000048,P070708081,1,338
20170101,1483228800,S12014,1172,1,ZZ000000000000,P071401022,1,2400
20170101,1483228800,S12029,112,1,CS029214000004,P060203001,1,98
20170101,1483228800,S12029,112,2,CS029214000004,P060701001,1,98
20170101,1483228800,S12029,1132,1,ZZ000000000000,P060104003,1,80
20170101,1483228800,S12029,1192,1,CS029414000005,P050102001,1,60


---

## 課題2
課題2-1，課題2-2のいずれかを選択し，結果を得るためのSQL文を書き，実行結果とともに示しなさい．

### 課題2-1: 月別売上の対昨年比

レシート明細テーブル（`receipt`）には2017年1月から2019年10月の間の購買情報が記録されている．2017年から2019年までの期間の売上を把握するために，1ヶ月ごとに以下の情報を集約表示せよ：
* 年月（`year_month`）
* 購買回数（`purchase_freq`）
* 月間売上高（`total_amount`）
* 購買1回あたりの平均購入額（`avg_amount_per_purchase`）
* 当該月の前年売上高（`total_amount_last_year`）
* 売上の対前年比（`ratio`）

※ 以下は，SQL実行結果のイメージである：

| year_month | purchase_freq | total_amount | avg_amount_per_purchase | total_amount_last_year | ratio |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 2017-01 | 1405 | 902056 | 642 | NULL | NULL  |
| 2017-02 | 1219 | 764413 | 627 | NULL | NULL  |
| ... | ... | ... | ... | ... | ... |
| 2017-12 | 1467 | 939654 | 640 | NULL | NULL  |
| 2018-01 | 1473 | 944509 | 641 | 902056 | 1.05 |
| 2018-02 | 1387 | 864128 | 623 | 764413 | 1.13 |
| ... | ... | ... | ... | ... | ... |
| 2019-10 | 1750 | 1143062 | 653 | 1069939 | 1.07 |

In [ ]:
%%sql


### 課題2-2: ABC分析

ABC分析は販売戦略を考えるために，売上によって商品をランク付けする手法である．
一般に，ABC分析では売上総額の
* 上位0〜70％の商品をAランク
* 上位70〜90％の商品をBランク
* 上位90〜100%の商品をCランク

とするランク付けを行う．

商品カテゴリ「菓子（`category_major_cd = 08`）」に属する商品について，小区分（`category_small_name`）ごとに売上を集計し，菓子カテゴリの売上総額に占める割合（構成比）を計算せよ．
また，売上がN位の小区分の行には売上額上位N位までの構成比累計，および構成比累計に基づくABC分析のランク付け結果も表示せよ．

※ 以下は，SQL実行結果のイメージである：

| カテゴリ名 | 売上 | 構成比 | 累積構成比 | ランク |
| ---- | ---- | ---- | ---- | ---- |
| 煎餅 | 194211 | 0.1306 | 0.1306 | A |
| 菓子詰め合わせ | 129580 | 0.0871 | 0.2178 | A |
| ナッツ類 | 129020 | 0.0867 | 0.3046 | A |
| ... | ... | ... | ... | ... |
| キャンディー | 46177 | 0.0310 | 0.7208 | B |
| ... | ... | ... | ... | ... |
| その他珍味 | 24086 | 0.0162 | 0.9073 | C |
| ... | ... | ... | ... | ... |

In [ ]:
%%sql


---
## ノック

### Knock 41: 結合1
レシート明細テーブル（`receipt`）と店舗テーブル（`store`）を内部結合し，レシート明細テーブルの全項目と店舗テーブルの店舗名（`store_name`）を10件表示させよ．

In [ ]:
%%sql


### Knock 42: 結合2
商品テーブル（`product`）とカテゴリテーブル（`category`）を内部結合し，商品テーブルの全項目とカテゴリテーブルの小区分名（`category_small_name`）を10件表示させよ．

In [ ]:
%%sql


### Knock 43: 結合3
顧客テーブル（`customer`）とレシート明細テーブル（`receipt`）から，各顧客ごとの売上金額合計を求めよ．ただし，売上実績がない顧客については売上金額を0として表示させること．また，顧客は性別コード（`gender_cd`）が女性（1）であるものを対象とし，非会員（顧客IDが"Z"から始まるもの）は除外すること．なお，表示項目は顧客ID（`customer_id`），顧客名（`customer_name`），売上金額合計とせよ．結果は10件だけ表示させればよい．

※ヒント：`IFNULL（変数1, 0）`関数は，変数1が空値（None）のとき0に置き換える

In [ ]:
%%sql


### Knock 44: 結合4

店舗と商品を組み合わせが全部で何通り「あり得るか」を計算したい．店舗（`store`）と商品（`product`）を直積を求めることで組合せ数を計算せよ．

In [ ]:
%%sql


### Knock 45: 結合5（難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，その結果を以下の3つを項目（列）として持つ表を出力せよ．
* 日付（`sales_ymd`）
* その日の売上金額（`total_amount`）
* その日の1週間前の売上金額（`one_week_before`）

なお，計算結果は10件表示すればよい．このノックはウィンドウ関数の`LAG`関数を使わずに取り組むこと．

* ヒント1：WITH句を使って集計結果のテーブルを一時的に保存する
* ヒント2：Window関数のROW_NUMBER関数を使う

In [ ]:
%%sql


### Knock 46: 結合6

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，前日からの売上金額増減を計算せよ．前日の売上金額の取得にはWindow関数の`LAG`関数を用いること．計算結果は10件表示すればよい．

In [ ]:
%%sql


### Knock 47: 結合7（難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，前日からの売上金額増減を計算せよ．なお，計算結果は10件表示すればよい．このノックはウィンドウ関数の`LAG`関数を使わずに取り組むこと．

※ヒント1：WITH句を使って集計結果のテーブルを一時的に保存する
※ヒント2：Window関数のROW_NUMBER関数を使う

In [ ]:
%%sql


### Knock 48: 欠損値1

商品テーブル（`product`）には単価（`unit_price`）と原価（`unit_cost`）が欠損しているレコードが存在する．そのようなレコードを表示せよ．

In [ ]:
%%sql


### Knock 49: 欠損値2（難）

商品テーブル（`product`）には単価（`unit_price`）と原価（`unit_cost`）が欠損しているレコードが存在する．単価と原価の欠損値について，それぞれの平均値で補完した商品テーブル`product_modified`として新たに生成せよ．

※備考: すでに存在するテーブルを削除するには`DROP TABLE IF EXISTS テーブル名`とすればよい．

In [ ]:
%%sql


### Knock 50: 副問い合わせ再訪

Knock48で生成した商品テーブル（`product_modified`）内のレコードのうち，元の商品テーブル（`product`）で単価と原価が欠損していた商品のレコードのみを表示せよ．なお，このノックでは該当する商品コードを`WHERE句`内で直接指定しないこと．また単価と原価は小数点第1位を四捨五入して表示すること．

In [ ]:
%%sql


### Knock 51: 四則演算1（SQLite固有）

商品テーブル（`product`）の単価（`unit_price`）と原価（`unit_cost`）から，各商品の利益（`profit`）および利益率（`profit_rate`）を算出せよ．
ただし，単価と原価にはNULL値が存在することに注意せよ．なお結果は10件表示させればよい．

※ヒント: SQLiteでは浮動小数点型への変換は`CAST(数値 AS REAL)`という関数で行う．MySQLではキャストを行わなくても浮動小数点演算が可能．

In [ ]:
%%sql


### Knock 52: 四則演算2

商品テーブル（`product`）の各商品について，利益率が30%となる新たな単価（`new_unit_price`）を求めよ．その際，1円未満を丸めること（四捨五入または偶数への丸めでよい）．ただし，単価と原価にはNULLが存在することに注意せよ．なお，結果を10件表示すればよい．

In [ ]:
%%sql


### Knock 53: 四則演算と集約演算1

レシート明細テーブル（`receipt`）と顧客テーブル（`customer`）を結合し，性別（`gender`）と年代（`age`から計算）ごとの売上金額（`amount`）の合計値を表示せよ．ただし，表示項目の構成は性別，年代，売上総額の3項目とすること．また，年代は10歳ごとの階級とすること．

In [ ]:
%%sql


### Knock 54: 四則演算と集約演算2（やや難）

レシート明細テーブル（`receipt`）と商品テーブル（`product`）を結合し，顧客毎に全商品の売上金額合計とカテゴリ大区分（`category_major_cd`）が`"07"`（瓶詰缶詰）の売上金額合計を計算の上，両者の比率を求めよ．
抽出対象はカテゴリ大区分`"07"`（瓶詰缶詰）の売上実績がある顧客のみとし，結果は10件表示させればよい．

※ヒント: Window関数の`OVER`句の中で`PARTION BY`を使う

In [ ]:
%%sql


### Knock 55: 四則演算と集約演算3（やや難）

レシート明細テーブル（`receipt`）を用いて顧客ID（`customer_id`）ごとの総売上を計算し，その上位10件を表示せよ．
その際，売上順位がN位の顧客IDの行には売上額上位N位までの顧客の累積売上額を併せて表示すること．
なお，顧客IDが"Z"から始まるのものは非会員を表すため，除外して計算すること．

※ヒント: Window関数の`OVER`句の中で`ORDER BY`を使うと…

In [ ]:
%%sql


### Knock 56: 2値化

顧客テーブル（`customer`）を東京（先頭3桁が100〜209のもの）とそれ以外の地域に分け，レシート明細テーブル（`receipt`）と結合し，全期間における購買頻度（`receipt_no`の総数）および売上総額を集計せよ．

※ヒント: `CASE`句を使う

In [ ]:
%%sql


### Knock 57: カテゴリ化1

顧客テーブル（`customer`）の年齢（`age`）をもとに10歳刻みで年代を算出し，年代ごとの顧客人数を集計せよ．なお，年代は10代であれば`1`，20代であれば`2`のようにすればよい．年代を表すカテゴリ名は任意とする．

※ 小数を整数に変換する必要があれば`CAST`関数を使うこと

In [ ]:
%%sql


### Knock 58: カテゴリ化2（やや難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を顧客ID（`customer_id`）ごとに合計し，顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し，顧客ID、売上金額合計とともに表示せよ．なお，表示件数は10件でよい．
カテゴリ値は以下のようにすること．
- 最小値以上第1四分位未満: 1を付与
- 第1四分位以上第2四分位未満: 2を付与
- 第2四分位以上第3四分位未満: 3を付与
- 第3四分位以上: 4を付与

※ ヒント: `PERCENT_RANK`関数を使う

In [ ]:
%%sql


### Knock 59: 日付型1

レシート明細テーブル（`receipt`）の売上日（`sales_epoch`）は購買行動が行われた日付が[UNIX時刻](https://ja.wikipedia.org/wiki/UNIX時間)で格納されている．`sales_epoch`の数値を`DATETIME`関数によって日付時刻オブジェクトに変換し，2018年のレシート明細のみを表示せよ．表示件数は10件でよい．

※ヒント: SQLiteデータベースでは`strftime`関数を使うことで，日付時刻オブジェクトから年や月，日を取り出すことができる．MySQLデータベース等では`DATE_FORMAT`関数を使うことで同様の操作ができる．

In [ ]:
%%sql


### Knock 60: 日付型2

レシート明細テーブル（`receipt`）に格納された売上金額（`amount`）を年月ごとに集約して表示せよ．すなわち，ある年のある月の売上総額が分かるように結果を表示せよ．なお，表示件数は先頭の10件でよい．

※ヒント: SQLiteデータベースでは`strftime`関数を使うことで，日付時刻オブジェクトから年や月，日を取り出すことができる．MySQLデータベース等では`DATE_FORMAT`関数を使うことで同様の操作ができる．

In [ ]:
%%sql
